In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [4]:
#Function to get Product Title

def get_title(soup):
    
    try:
        
        title = soup.find("span", attrs={'id':'productTitle'})
    
        title_text = title.text
        
        title_string = title_text.strip()
        
    except AttributeError:
        title_string = ""

    return title_string

def get_price(soup):
    try:
        price = soup.find("span", attrs={'class':"a-offscreen"})
        
        price_text = price.text.strip()
        
    except AttributeError:
        price_text = ""
        
    return price_text


def get_rating(soup):
    try:
        rating = soup.find("span", attrs={'class':"a-icon-alt"}).text.strip()
        
    except AttributeError:
        rating = ""
        
    return rating

def get_reviews(soup):
    try:
        review = soup.find("span", attrs={'id':"acrCustomerReviewText"}).text.strip()
        
    except AttributeError:
        review = ""
        
    return review

def get_availability(soup):
    try:
        availibility = soup.find("span", attrs={'class':"a-size-medium a-color-success"}).text.strip()
        
    except AttributeError:
        availibility = ""
        
    return availibility



In [8]:
 if __name__=='__main__':
    
    #define URL
    URL = "https://www.amazon.com/s?k=chocolate&crid=E925NQPL9UCV&sprefix=choc%2Caps%2C270&ref=nb_sb_ss_ts-doa-p_2_4"
    
    #define headers for request
    Headers = {'User-Agent':'https://www.amazon.com/s?k=chocolate&crid=E925NQPL9UCV&sprefix=choc%2Caps%2C270&ref=nb_sb_ss_ts-doa-p_2_4','Accept-Language':'en-US, en;q=0.5'}
    
    #HTTP request
    webpage = requests.get(URL,headers=Headers)
    
    #Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    #Fetch the links
    links = soup.find_all("a", attrs={'class':"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})
    
    #List to store individual links
    links_list = []
    
    #Store the links
    for link in links:
        links_list.append("https://amazon.com" + link.get('href'))
    
    #create a dictionary to store product details
    d = {"title":[], "price":[], "rating":[], "reviews":[], "availability":[]}
    
    #Iterating to get the product information
    for link in links_list:
        new_web = requests.get(link,headers=Headers)
        
        new_soup = BeautifulSoup(new_web.content,"html.parser")
        
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_reviews(new_soup))
        d['availability'].append(get_availability(new_soup))
        
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan , inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv",header=True, index=False)
    

In [9]:
amazon_df

,title,price,rating,reviews,availability
0,Valentines Gift Basket for Women | Chocolate G...,$35.99,4.5 out of 5 stars,383 ratings,In Stock.
1,"REESE'S, HERSHEY'S and KIT KAT Milk Chocolate ...",$17.58,4.7 out of 5 stars,"27,679 ratings",In Stock
2,"Chocolate Gift Basket, Valentines Day Food Gif...",$35.99,4.6 out of 5 stars,"9,858 ratings",In Stock.
3,REESE'S Miniatures Milk Chocolate and Peanut B...,$11.98,4.7 out of 5 stars,"25,405 ratings",In Stock.
4,"Glico Pocky, Chocolate Cream Covered Biscuit S...",$3.83,4.5 out of 5 stars,"21,638 ratings",In Stock
...,...,...,...,...,...
72,Theo Chocolate Hazelnut Crisp Organic Milk Cho...,$2.70,4.5 out of 5 stars,310 ratings,In Stock.
73,"Bixby Bar Milk Chocolate Butter, 4.2 oz",$1.26,4.7 out of 5 stars,28 ratings,In Stock.
74,UNREAL Dark Chocolate Coconut Bars | 3g Sugar ...,$14.99,4.7 out of 5 stars,"4,093 ratings",In Stock.
75,"HERSHEY'S Milk Chocolate XL Candy, Bulk, Glute...",$23.76,4.7 out of 5 stars,"7,346 ratings",In Stock.
